# Hello, TensorFlow!

* Names and execution in Python and TensorFlow
* The simplest TensorFlow graph
* The simplest TensorFlow neuron
* See your graph in TensorBoard
* Making the neuron learn
* Training diagnostics in TensorBoard
* Flowing onward

*See the [O'Reilly post](https://www.oreilly.com/learning/hello-tensorflow) for more explanatory text.*

---

### Names and execution in Python and TensorFlow

![An object has no name.](img/an_object_has_no_name.jpg)

*Image courtesy of [Hadley Wickham](https://twitter.com/hadleywickham/status/732288980549390336).*

In [ ]:
foo = []

In [ ]:
bar = foo

In [ ]:
foo == bar

In [ ]:
foo is bar

In [ ]:
id(foo)

In [ ]:
id(bar)

In [ ]:
foo.append(bar)

In [ ]:
foo

![foo points to itself](img/foo_loop.png)

*Image made with [draw.io](https://draw.io/).*

---

### The simplest TensorFlow graph

In [ ]:
import tensorflow as tf

In [ ]:
graph = tf.get_default_graph()

In [ ]:
graph.get_operations()

In [ ]:
input_value = tf.constant(1.0)

In [ ]:
operations = graph.get_operations()

In [ ]:
operations

In [ ]:
operations[0].node_def

In [ ]:
input_value

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(input_value)

### The simplest TensorFlow neuron

In [ ]:
weight = tf.Variable(0.8)

In [ ]:
for op in graph.get_operations(): print(op.name)

(The parentheses for `print` are necessary for Python 3 but optional for Python 2.)

In [ ]:
output_value = weight * input_value

In [ ]:
op = graph.get_operations()[-1]

In [ ]:
op.name

In [ ]:
for op_input in op.inputs: print(op_input)

In [ ]:
init = tf.initialize_all_variables()

In [ ]:
sess.run(init)

In [ ]:
sess.run(output_value)

This is the neuron's "inference" or "forward pass".

---

### See your graph in TensorBoard

In [ ]:
tf.reset_default_graph()
sess = tf.Session()
!rm -rf log_simple_graph
!rm -rf log_simple_stat

In [ ]:
x = tf.constant(1.0, name='input')
w = tf.Variable(0.8, name='weight')
y = tf.mul(w, x, name='output')

In [ ]:
summary_writer = tf.train.SummaryWriter('log_simple_graph', sess.graph)

In [ ]:
!tensorboard --logdir=log_simple_graph

After startup, go to http://localhost:6006/#graphs to see the interface.

To continue with the notebook, interrupt the kernel by using the square "stop" button or by typing `esc`, `i`, `i`.

### Making the neuron learn

In [ ]:
y_ = tf.constant(0.0)

In [ ]:
loss = (y - y_)**2

In [ ]:
optim = tf.train.GradientDescentOptimizer(learning_rate=0.025)

In [ ]:
grads_and_vars = optim.compute_gradients(loss)

In [ ]:
sess.run(tf.initialize_all_variables())

In [ ]:
sess.run(grads_and_vars[0][0])

We have $y=0.8$ and $y\_=0$.

So the loss is $(0.8 - 0)^2=0.64$.

And the derivative of the loss is $2(0.8 - 0)$.

In [ ]:
sess.run(optim.apply_gradients(grads_and_vars))

We have now completed a forward and backward pass.

In [ ]:
sess.run(w)

This comes from subtracting the learning rate times the derivative of the loss:

$0.8 - (0.025)(1.6) = 0.8 - 0.04 = 0.76$

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.025).minimize(loss)

In [ ]:
for i in range(100):
    sess.run(train_step)

In [ ]:
sess.run(y)

### Training diagnostics in TensorBoard

In [ ]:
sess.run(tf.initialize_all_variables())

In [ ]:
for i in range(100):
    print('before step {}, y is {}'.format(i, sess.run(y)))
    sess.run(train_step)

In [ ]:
summary_y = tf.scalar_summary('output', y)

In [ ]:
summary_writer = tf.train.SummaryWriter('log_simple_stat')

In [ ]:
sess.run(tf.initialize_all_variables())
for i in range(100):
    summary_str = sess.run(summary_y)
    summary_writer.add_summary(summary_str, i)
    sess.run(train_step)

In [ ]:
!tensorboard --logdir=log_simple_stat

After startup, go to http://localhost:6006/#events to see the interface.

To continue with the notebook, interrupt the kernel by using the square "stop" button or by typing `esc`, `i`, `i`.

---

### Flowing onward

In [ ]:
tf.reset_default_graph()
!rm -rf log_simple_stats

The script below is designed to stand alone, and will also work here with the default graph reset.

In [ ]:
import tensorflow as tf

x = tf.constant(1.0, name='input')
w = tf.Variable(0.8, name='weight')
y = tf.mul(w, x, name='output')
y_ = tf.constant(0.0, name='correct_value')
loss = tf.pow(y - y_, 2, name='loss')
train_step = tf.train.GradientDescentOptimizer(0.025).minimize(loss)

for value in [x, w, y, y_, loss]:
    tf.scalar_summary(value.op.name, value)

summaries = tf.merge_all_summaries()

sess = tf.Session()
summary_writer = tf.train.SummaryWriter('log_simple_stats', sess.graph)

sess.run(tf.initialize_all_variables())
for i in range(100):
    summary_writer.add_summary(sess.run(summaries), i)
    sess.run(train_step)

In [ ]:
!tensorboard --logdir=log_simple_stats

After startup, go to http://localhost:6006/#events to see the interface.

To continue with the notebook, interrupt the kernel by using the square "stop" button or by typing `esc`, `i`, `i`.